<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [135]:
def dswap_test_circuit(qc1: QuantumCircuit, qc2: QuantumCircuit) -> QuantumCircuit:
    """ Construct the destructive SWAP test circuit given two circuits.

    Args:
        qc1(qiskit.QuantumCircuit): Quantum circuit for the 
        first state.
        qc2(qiskit.QuantumCircuit): Quantum circuit for the 
        second state.
    Output:
        (qiskit.QuantumCircuit): swap test circuit.
    """
    # Helper variables
    n_total = qc1.num_qubits + qc2.num_qubits
    range_qc1 = [i  for i in range(qc1.num_qubits)]
    range_qc2 = [i + qc1.num_qubits for i in range(qc2.num_qubits)]
    
    # Constructing the SWAP test circuit
    qc_swap = QuantumCircuit(n_total , n_total)
    qc_swap.append(qc1, range_qc1)
    qc_swap.append(qc2, range_qc2)
    
    
    for index, qubit in enumerate(range_qc1):
        qc_swap.cx(qubit, range_qc2[index])
        qc_swap.h(qubit)
    
    
    for index, qubit in enumerate(range_qc1):
        qc_swap.measure(qubit, 2*index)
        
    for index, qubit in enumerate(range_qc2):
        qc_swap.measure(range_qc2[index] , 2*index + 1)
    return qc_swap

In [138]:
import textwrap
import qiskit
from qiskit import QuantumCircuit, execute
from typing import Union
from qiskit.aqua import QuantumInstance
from qiskit.providers import BaseBackend

def measure_dswap_test(qc1: QuantumCircuit, qc2: QuantumCircuit,
                     backend: Union[BaseBackend,QuantumInstance],
                     num_shots: int=10000) -> float:
    """Returns the fidelity from a destructive SWAP test.

    Args:
        qc1 (QuantumCircuit): Quantum Circuit for the first state.
        qc2 (QuantumCircuit): Quantum Circuit for the second state.
        backend (Union[BaseBackend,QuantumInstance]): Backend.
        num_shots (int, optional): Number of shots. Defaults to 10000.

    Returns:
        float: result of the overlap betweeen the first and second state.
    """
    n = qc1.num_qubits 
    swap_circuit = dswap_test_circuit(qc1, qc2)

    # Check if the backend is a quantum instance.
    if qiskit.aqua.quantum_instance.QuantumInstance == type(backend):
        count = backend.execute(swap_circuit).get_counts()
    else:
        count = execute(swap_circuit, backend=backend, shots=num_shots).result().get_counts()
    
    result = 0
    for meas, counts in count.items():
        split_meas = textwrap.wrap(meas, 2)
        for m in split_meas:
            if m == '11':
                result -= counts
            else:
                result += counts

    total = sum(count.values())
    
    return result/(n*total)

In [139]:
from qiskit import QuantumCircuit, execute

qc1 = QuantumCircuit(n)
qc2 = QuantumCircuit(n)

backend = BasicAer.get_backend('qasm_simulator')
measure_dswap_test(qc1, qc2, backend)

1.0

In [140]:
from qiskit import BasicAer
backend = BasicAer.get_backend('qasm_simulator')
num_shots = 10000
count = execute(swap_circuit, backend=backend, shots=num_shots).result().get_counts()

In [127]:
import textwrap
result = 0
for meas, counts in count.items():
    split_meas = textwrap.wrap(meas, 2)
    for m in split_meas:
        if m == '11':
            result -= counts
        else:
            result += counts

total = sum(count.values())
result/(n*total)

1.0